In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("spam_processed.csv")
data

,preprocessed_text,v1
0,go jurong point crazi avail bugi n great world...,0
1,ok lar joke wif u oni,0
2,free entri 2 wkli comp win fa cup final tkt 21...,1
3,u dun say earli hor u c alreadi say,0
4,nah think goe usf live around though,0
...,...,...
5164,2nd time tri 2 contact u pound prize 2 claim e...,1
5165,b go esplanad fr home,0
5166,piti mood suggest,0
5167,guy bitch act like interest buy someth els nex...,0


In [3]:
data.isna().sum()

preprocessed_text    9
v1                   0
dtype: int64

In [4]:
data['preprocessed_text'] = data['preprocessed_text'].fillna('')
data['preprocessed_text'] = data['preprocessed_text'].astype(str)

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf = TfidfVectorizer(max_features=3000)
X = tfidf.fit_transform(data['preprocessed_text']).toarray()
y = data['v1'].values

y.shape

(5169,)

In [12]:
from sklearn.model_selection import train_test_split    


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
import tensorflow as tf
import keras
from keras.api.models import Sequential
from keras.api.layers import Dense

In [21]:
model = Sequential([
    keras.Input(shape=(3000,)),
    Dense(units=12, activation='relu'),
    Dense(units=10, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [22]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

In [23]:
history = model.fit(X, y, epochs=15, batch_size=32, validation_data=(X_test, y_test), callbacks=[callback])

Epoch 1/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.8282 - loss: 0.6091 - val_accuracy: 0.9333 - val_loss: 0.2606
Epoch 2/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9516 - loss: 0.1920 - val_accuracy: 0.9836 - val_loss: 0.0743
Epoch 3/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9843 - loss: 0.0655 - val_accuracy: 0.9865 - val_loss: 0.0425
Epoch 4/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9896 - loss: 0.0407 - val_accuracy: 0.9942 - val_loss: 0.0299
Epoch 5/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9963 - loss: 0.0217 - val_accuracy: 0.9942 - val_loss: 0.0233
Epoch 6/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9965 - loss: 0.0211 - val_accuracy: 0.9952 - val_loss: 0.0184
Epoch 7/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9964 - loss: 0.0162 - val_accuracy: 0.9971 - val_loss: 0.0149
Epoch 8/15
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9974 - loss: 0.0118 - val_accuracy: 0.

In [25]:
from sklearn.metrics import accuracy_score, f1_score

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
print("accuracy_score: ", accuracy_score(y_test, y_pred))
print("f1_score: ", f1_score(y_test, y_pred))

33/33 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
accuracy_score:  0.9990328820116054
f1_score:  0.9965397923875432


In [26]:
import pickle

pickle.dump(model, open('ann.pkl', 'wb'))